In [ ]:
import json
import nltk
from spacy import detect 
import re 
import spacy
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

In [ ]:
f=open("training.json")
training_set=json.load(f)

f2 = open("test.json")
test_set = json.load(f2)

In [ ]:
l_training=[]
for i in range(len(training_set)):
    l_training.append({"text":training_set[i]["data"]["text"], "results":training_set[i]["predictions"][0]["result"]})

l_test = []
for i in range(len(test_set)):
    l_test.append({"text":test_set[i]["data"]["text"], "results":test_set[i]["predictions"][0]["result"]})

In [ ]:
l_neg=[]
l_unc=[]
l_nsco=[]
l_usco=[]
for dic in l_training:
    for dic2 in dic["results"]:
        if(dic2["value"]["labels"][0]=="NEG"):
            l_neg.append(dic["text"][dic2["value"]["start"]:dic2["value"]["end"]])
        
        elif(dic2["value"]["labels"][0]=="UNC"):
            l_unc.append(dic["text"][dic2["value"]["start"]:dic2["value"]["end"]])
        
        elif(dic2["value"]["labels"][0]=="NSCO"):
            l_nsco.append(dic["text"][dic2["value"]["start"]:dic2["value"]["end"]])
        
        elif(dic2["value"]["labels"][0]=="USCO"):
            l_usco.append(dic["text"][dic2["value"]["start"]:dic2["value"]["end"]])

In [ ]:
for i in range(len(l_neg)):
    if(not l_neg[i][0].isalpha()):
        l_neg[i]=l_neg[i][1:]
    if(not l_neg[i][-1].isalpha()):
        l_neg[i]=l_neg[i][:-1]

for i in range(len(l_unc)):
    if(not l_unc[i][0].isalpha()):
        l_unc[i]=l_unc[i][1:]
    if(not l_unc[i][-1].isalpha()):
        l_unc[i]=l_unc[i][:-1]

for i in range(len(l_nsco)):
    if(not l_nsco[i][0].isalpha()):
        l_nsco[i]=l_nsco[i][1:]
    if(not l_nsco[i][-1].isalpha()):
        l_nsco[i]=l_nsco[i][:-1]

for i in range(len(l_usco)):
    if(not l_usco[i][0].isalpha()):
        l_usco[i]=l_usco[i][1:]
    if(not l_usco[i][-1].isalpha()):
        l_usco[i]=l_usco[i][:-1]

In [ ]:
l_neg=list(set(l_neg))
l_unc=list(set(l_unc))
l_nsco=list(set(l_nsco))
l_usco=list(set(l_usco))

In [ ]:
def search_words(list_words, text):
    pattern = '|'.join(re.escape(palabra) for palabra in list_words)
    pattern= r"\b(" +pattern+ r")\b"
    
    coincidencias = []
    ini = []
    end = []
    
    # Finding all occurrences of any word in the text
    for match in re.finditer(pattern, text):
        coincidencias.append(match.group(0))
        ini.append(match.start())
        end.append(match.end())    
    return coincidencias, end, ini

In [ ]:
nlp = spacy.load("es_core_news_md")

# Function to extract medical terms.
def extract_medical_terms(text):
    doc = nlp(text)
    medical_terms = []
    for token in doc:
        if token.pos_ == "NOUN" or token.pos_ == "ADJ":
            medical_terms.append(token.text)
    return medical_terms

l_medical_term2 = []

for sentence in l_nsco:
    medical_terms = extract_medical_terms(sentence)
    l_medical_term2.append(medical_terms)

for sentence in l_usco:
    medical_terms = extract_medical_terms(sentence)
    l_medical_term2.append(medical_terms)

l_medical_terms =[]
for x in range (len(l_medical_term2)):
    for y in range(len(l_medical_term2[x])):
        l_medical_terms.append(l_medical_term2[x][y])
        

In [ ]:
l_medical_terms = list(set(l_medical_terms))

In [ ]:
def eliminate_patient_information(string):
    index = string.find("informe")
    if index != -1:
        return string[index:]
    else:
        return string

In [ ]:
# Auxiliary Function
def find_scope_baseline(target_word, sentence, l_medical):
    """
    
    It returns False in case there is not a medical term near.

    Otherwise it returns the ini and end indexes of the scope.

    """

    language = detect(sentence)
    if language == 'ca': 
        nlp = spacy.load("ca_core_news_sm")
    elif language == 'es':
        nlp = spacy.load("es_core_news_sm")
    else:
        nlp = spacy.load("es_core_news_sm")

    doc = nlp(sentence)

    keyword_token = None
    for token in doc:
        if token.text.lower() == target_word.lower():
            keyword_token = token
            break
    
    if not keyword_token:
        return False, False

    nearest_medical_term = None
    nearest_distance = float('inf')


    for token in doc:
        if token.text.lower() in l_medical:         
            distance = abs(token.i - keyword_token.i)
            if distance < nearest_distance:
                nearest_medical_term = token.text
                nearest_distance = distance


    keyword_index = keyword_token.i 
    
    if nearest_medical_term == None:
        return False,False
    medical_index = [token.i for token in doc if token.text.lower() == nearest_medical_term.lower()][0]   
    
    if keyword_index < medical_index:
        ini_index = keyword_index + 1
        end_index = medical_index 
    else:
        ini_index = medical_index 
        end_index = keyword_index -1


    # Obtener el texto del "scope"
    scope_text = doc[ini_index:end_index + 1].text

    inicio = sentence.find(scope_text)
    fin = inicio + len(scope_text)

    return inicio, fin

target_word= 'no'
sentence = "El personaje no tiene madera de "
l_medical = ['paciente']
a,b = find_scope_baseline(target_word, sentence, l_medical)


In [ ]:
# Auxiliary Function
def find_scope_baseline(target_word, sentence, l_medical):
    """
    
    It returns False in case there is not a medical term near.

    Otherwise it returns the ini and end indexes of the scope.

    """

    language = detect(sentence)
    if language == 'ca': 
        nlp = spacy.load("ca_core_news_sm")
    elif language == 'es':
        nlp = spacy.load("es_core_news_sm")
    else:
        nlp = spacy.load("es_core_news_sm")

    doc = nlp(sentence)

    keyword_token = None
    for token in doc:
        if token.text.lower() == target_word.lower():
            keyword_token = token
            break
    
    if not keyword_token:
        return False, False

    nearest_medical_term = None
    nearest_distance = float('inf')


    for token in doc:
        if token.text.lower() in l_medical:         
            distance = abs(token.i - keyword_token.i)
            if distance < nearest_distance:
                nearest_medical_term = token.text
                nearest_distance = distance


    keyword_index = keyword_token.i 
    
    if nearest_medical_term == None:
        return False,False
    medical_index = [token.i for token in doc if token.text.lower() == nearest_medical_term.lower()][0]   
    
    if keyword_index < medical_index:
        ini_index = keyword_index + 1
        end_index = medical_index 
    else:
        ini_index = medical_index 
        end_index = keyword_index -1


    # Obtener el texto del "scope"
    scope_text = doc[ini_index:end_index + 1].text

    inicio = sentence.find(scope_text)
    fin = inicio + len(scope_text)

    return inicio, fin

target_word= 'no'
sentence = "El personaje no tiene madera de "
l_medical = ['paciente']
a,b = find_scope_baseline(target_word, sentence, l_medical)

In [ ]:
import re
mini_prueba = l_training[:10]
print(len(mini_prueba))

l_res = [] # List to store the final results. 

keywords_voc = l_neg + l_unc
for element in mini_prueba:
    text = eliminate_patient_information(element['text'])
    counter = len(element['text']) - len(text)
    sentences = nltk.sent_tokenize(text)
    scopes = []
    dictionary_res_list = []

    for sentence in sentences:
        keywords_appear, end_keyword, start_keyword = search_words(keywords_voc,sentence)

        for i in range(len(keywords_appear)):
            if keywords_appear[i] in l_neg:
                label = ["NEG"]
                res = {"value":{"start": start_keyword[i] + counter, "end": end_keyword[i] + counter, "labels": label}}
                dictionary_res_list.append(res)

                ini,end = find_scope_baseline(keywords_appear[i], sentence, l_medical_terms)
                if ini != False:
                    label = ["NSCO"]
                    res = {"value":{"start": ini + counter, "end": end + counter, "labels": label}}
                    dictionary_res_list.append(res)

            else:

                label = ["UNC"]
                ini,end = find_scope_baseline(keywords_appear[i], sentence, l_medical_terms)
                res = {"value":{"start": start_keyword[i] + counter, "end": end_keyword[i] + counter, "labels": label}}
                dictionary_res_list.append(res)

                ini,end = find_scope_baseline(keywords_appear[i], sentence, l_medical_terms)
                if ini != False:
                    label = ["USCO"]
                    res = {"value":{"start": ini + counter, "end": end + counter, "labels": label}}
                    dictionary_res_list.append(res)

        counter+=len(sentence)
                    

    l_res.append({'text':element['text'], 'result': dictionary_res_list})
        
print(l_res)


In [ ]:
# Save the resulting json
with open("res_training_10_Simplification.json", "w") as archivo:
    json.dump(l_res, archivo)